# basicsynbio Addgene submission collection

This Notebook contains code to aid the submission of basicsynbio linkers to addgene.

## Notes on data preperation

- [x] The p15A plasmid supplied by Marko `BS_x6x` was corrected given it has a "t" insertion.
- [x] The Benchling feature library (BASIC_SEVA_benchling_misc_features_library.csv) was updated commit: `a2fb18c`.

## Aims and objectives for cell/s below

- [x] Import the following objects:
  - [x] pickled BasicLinker `BSEVA_L1`
  - [x] Selected Oris and AbR markers specified in #130.
  - [x] mScarlet counter selection casettes for associated Oris.
- [x] Assemble BS_x7x_TS gblocks with BS_1xx to place them in storage vectors. Export associated data. 
- [x] Generate a dictionary mapping between Oris and counter-selection casettes.
- [x] Make assemblies and export the following data:
  - [x] Genbank file containing all correctly assembled backbones.
  - [x] Manual and automated build instructions.
- [x] Calculate mass of AbR markers to add to clip reaction. *pJet contains multiple BsaI sites*.

## Notes on implementation of above

- Following commit ... x9x constructs were assembled with the 405 counter-selection cassette instead of the 407 cassette. This was in an attempt to lower the burden of these constructs.

In [11]:
import basicsynbio as bsb
from basicsynbio.cam import seqrecord_hexdigest
from basicsynbio.utils import MARKER_DICT, ORI_DICT
from Bio import SeqIO
import pandas as pd
from pathlib import Path
import pickle
import re

In [12]:
path_to_seqs = Path.cwd().parents[0] / "sequences"
path_to_initial_mods = path_to_seqs / "genbank_files" / "BASIC_SEVA_collection" / "initial_mods"
# Import objects
linkers = bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]
with open(path_to_seqs / "alternative_formats" / "pickles" / "SEVA-BB1", "rb") as file:
    bb_linker = pickle.load(file)
abr_markers = list(
    bsb.import_parts(path_to_initial_mods / "AbR_markers.gb",
    "genbank"
))
oris = list(
    bsb.import_parts(path_to_initial_mods / "oris.gb",
    "genbank"
))
cs_cassettes = bsb.import_parts(path_to_initial_mods / "mScarlet_cs_cassettes.gb",
    "genbank"
)
cs_cassettes = [
    bsb.BasicAssembly(cs_cassette.id, linkers["LMP"], cs_cassette, linkers["LMS"], bsb.BASIC_SEVA_PARTS["v0.1"]["18"]).return_part(
        id=cs_cassette.id,
        name=cs_cassette.id,
    ) for cs_cassette in cs_cassettes
]
ts_ori_gblocks = list(bsb.import_parts(path_to_initial_mods / "TS_ori_gblocks.gb", "genbank"))
# Check for unwanted Benchling annotations
initial_modules = abr_markers + oris + cs_cassettes
unwanted_annotations = ("Translation", "Primer")
for initial_module in initial_modules:
    if initial_module.id == "BS_5axx":
        continue
    qualifier_values = [value[0] for feature in initial_module.features for value in list(feature.qualifiers.values())]
    for unwanted_annotation in unwanted_annotations:
        re_matches = [re.match(unwanted_annotation, value[:len(unwanted_annotation)]) for value in qualifier_values]
        if re_matches.count(None) != len(re_matches):
            raise ValueError(f"{initial_module.id} contains unwanted annotations")

In [13]:
# Assemble BS_x7x_TS
ts_ori_assemblies = [bsb.BasicAssembly(
    ts_ori.id,
    linkers["LMP"],
    ts_ori,
    linkers["LMS"],
    [marker for marker in abr_markers if marker.id == "BS_1xx"][0]
) for ts_ori in ts_ori_gblocks]
bsb.export_sequences_to_file(
    ts_ori_assemblies,
    path_to_initial_mods / "TS_ori_modules.gb"
)
bsb.pdf_instructions(
    bsb.BasicBuild(*ts_ori_assemblies),
    "/home/hainesm6/github_repos/LondonBiofoundry/basicsynbio/pdfs/bs_x7x_ts_oris_instructions.pdf",
)

/home/hainesm6/.pyenv/versions/3.8.5/envs/basicsynbio/lib/python3.8/site-packages/Bio/SeqIO/InsdcIO.py:726: BiopythonWarning: Increasing length of locus line to allow long name. This will result in fields that are not in usual positions.
  warnings.warn(


'/home/hainesm6/github_repos/LondonBiofoundry/basicsynbio/pdfs/bs_x7x_ts_oris_instructions.pdf'

In [14]:
def bseva_assembly(
    marker: bsb.BasicPart,
    ori: bsb.BasicPart,
    cs_cassette: bsb.BasicPart,
    id: str ="foobar",
) -> bsb.BasicAssembly:
    """Return a BASIC SEVA backbone as an assembly object.

    Args:
        marker: Antibiotic resistance marker.
        ori: Origin of replication.
        cs_cassette: Counter-selection containing mScarlet CDS.
        id: ID for assembly   
    """
    return bsb.BasicAssembly(
        id,
        linkers["LMP"],
        marker,
        bb_linker,
        ori,
        linkers["LMS"],
        cs_cassette
    )


def marker_abbreviation(marker_id: str) -> str:
    """Return abbreviation for resistance marker."""
    if re.match("BS_\daxx", marker_id) != None:
        return abr_marker.id[-4:-2]
    return abr_marker.id[-3]


# Map oris with counter selection cassettes
ori_cs_cassette_mapping = {
    "BS_x5ax": "B407_J23119-RBS34-mScarl",
    "BS_x6x": "B407_J23119-RBS34-mScarl",
    "BS_x7x": "B408_J23119-RBS-A12-mSc",
    "BS_x7x_pKD46": "B408_J23119-RBS-A12-mSc",
    "BS_x9x": "B405_J23106-RBS34-mScarl",
}
ori_cs_cassette_mapping = {
    key: cs_cassettes[[cs_cassette.id for cs_cassette in cs_cassettes].index(value)] for key, value in ori_cs_cassette_mapping.items()
}
# Make assemblies and export data
MARKER_DICT = MARKER_DICT.copy()
MARKER_DICT["5a"] = "Tetracycline"
assemblies = []
for abr_marker in abr_markers:
    abr_abbreviation = marker_abbreviation(abr_marker.id)
    for ori in oris:    
        assembly = bseva_assembly(
            abr_marker,
            ori,
            ori_cs_cassette_mapping[ori.id]
        ) 
        if ori.id == "BS_x7x_pKD46":
            assembly.id = f"BASIC_SEVA_{abr_abbreviation}7_pKD46.10"
            assembly.description = f"BASIC SEVA vector containing {MARKER_DICT[abr_abbreviation]} resistance marker and temperature sensitive pSC101 origin of replication"
        elif re.match("BS_x\dax", ori.id):
            assembly.id = f"BASIC_SEVA_{abr_abbreviation + ori.id[-3:-1]}.10"
            assembly.description = f"BASIC SEVA vector containing {MARKER_DICT[abr_abbreviation]} resistance marker and {ORI_DICT[ori.id[-3]]} origin of replication"           
        else:
            assert re.match("BS_x\dx", ori.id) != None
            assembly.id = f"BASIC_SEVA_{abr_abbreviation + ori.id[-2]}.10"
            assembly.description = f"BASIC SEVA vector containing {MARKER_DICT[abr_abbreviation]} resistance marker and {ORI_DICT[ori.id[-2]]} origin of replication"
        assemblies.append(assembly)
build = bsb.BasicBuild(*assemblies)
bsb.export_sequences_to_file(
    (assembly.return_part(
        description=assembly.description,
        name=assembly.id,
        id=seqrecord_hexdigest(assembly.return_part())
    ) for assembly in build.basic_assemblies),
    Path.cwd().parents[0] / "basicsynbio" / "parts_linkers" / "BASIC_SEVA_collection_v10.gb"
)
bsb.export_echo_assembly(
    build,
    Path.cwd().parents[0] / "csv_xlsx_files" / "BASIC_SEVA_collection_v10_echo_scripts.zip",
    assemblies_per_clip=20,
    alternate_well=True,
)
bsb.pdf_instructions(
    build,
    "/home/hainesm6/github_repos/LondonBiofoundry/basicsynbio/pdfs/BASIC_SEVA_collection_v10_manual.pdf",
    assemblies_per_clip=20,
)


'/home/hainesm6/github_repos/LondonBiofoundry/basicsynbio/pdfs/BASIC_SEVA_collection_v10_manual.pdf'

In [15]:
# Calculate AbR marker mass to add to clip reaction
PJET_LEN = 2974
def approx_clip_mass(seq_length: int, ndigit: int =None) -> float:
    """Calculate the approximate mass of a BASIC part (ng) to add to a clip reaction."""
    return round(2.5*30e-6*(seq_length*607.4 + 157.9), ndigit)
abr_masses = {
    "Marker ID": [marker.id for marker in abr_markers],
    "mass (ng)": [
        approx_clip_mass(PJET_LEN + len(marker.seq)) for marker in abr_markers
    ],
}
abr_masses = pd.DataFrame(abr_masses)
abr_masses.to_csv(
    Path.cwd().parents[0] / "csv_xlsx_files" / "AbR_marker_clip_masses.csv",
    index=False,
)

## Results and discussion

- It has previously been reported that inhibition of protein synthesis dramatically increases the copy number of relaxed plasmids such as pBR322-ROP (Sambrook). This would lead to relatively higher expression of mScarlet compared to antibiotics that don't inhibit protein synthesis e.g. carbenicilin. This could explain why certain x9x constructs were burdensome.   
- Lowering the concentration of tetracycline to 5 microgram/mL may improve the growth rate of BASIC_SEVA_5a5a.10, 5a7 and presumably 5a7_pKD46.10. All these plasmids replicate via a stringent mechanism and therefore cannot increase copy number if protein expression is blocked. This could explain why dropping the concentration improved the growth of these plasmids.

